## Fuzzy Deduplication Workflow

This notebook shows how to:
1. Load config from `config.yml`.
2. Load and unify Dataset1 + Dataset2 (in Dataset1’s format).
3. Block and fuzzy-match records.
4. Merge duplicates.
5. Save final combined data.

In [ ]:
import pandas as pd
from src.fuzzy_helpers import (
    load_config,
    load_dataset1,
    load_dataset2_and_map,
    combine_dataframes,
    block_data,
    build_comparison_features,
    classify_duplicates,
    merge_duplicate_pairs
)

### 1. Load Configuration

Our `config.yml` file contains:
- dataset1_path
- dataset2_path
- output_path
- dataset1_columns
- dataset2_to_dataset1
- blocking_keys
- fuzzy_keys
- similarity_threshold

In [ ]:
config_path = "config.yml"  # Adjust if needed
config = load_config(config_path)

dataset1_path = config["dataset1_path"]
dataset2_path = config["dataset2_path"]
output_path = config["output_path"]

dataset1_columns = config["dataset1_columns"]
dataset2_to_dataset1 = config["dataset2_to_dataset1"]
blocking_keys = config["blocking_keys"]
fuzzy_keys = config["fuzzy_keys"]
similarity_threshold = float(config["similarity_threshold"])

### 2. Load Datasets

Dataset1 is presumably already in the final schema, but we ensure all columns exist.  
Dataset2 is mapped to that same schema using the provided dictionary.


In [ ]:
df1 = load_dataset1(dataset1_path, dataset1_columns)
df2_mapped = load_dataset2_and_map(dataset2_path, dataset1_columns, dataset2_to_dataset1)

### 3. Combine

In [ ]:
combined_df = combine_dataframes(df1, df2_mapped)

**Optional**: Some quick cleanup, e.g., filling NaNs with empty strings in text columns.


In [ ]:
for col in combined_df.columns:
    if combined_df[col].dtype == object:
        combined_df[col] = combined_df[col].fillna("")

### 4. Block Data

We create candidate pairs only for rows that share the same values in `blocking_keys`.

In [ ]:
candidate_pairs = block_data(combined_df, blocking_keys)

### 5. Build Fuzzy Comparison Features

Using jaro-winkler on the `fuzzy_keys` columns.

In [ ]:
features_df = build_comparison_features(combined_df, candidate_pairs, fuzzy_keys)
features_df.head()

### 6. Classify Duplicates

If the average similarity across these fields is >= `similarity_threshold`, 
they're flagged as duplicates.

In [ ]:
duplicates_idx = classify_duplicates(features_df, similarity_threshold)
print(f"Found {len(duplicates_idx)} pairs classified as duplicates.")

### 7. Merge Duplicate Pairs

We'll keep the earliest row from each connected group of duplicates.

In [ ]:
deduped_df = merge_duplicate_pairs(combined_df, duplicates_idx)
print(f"Deduplicated dataframe has {len(deduped_df)} rows.")

### 8. Save Final

In [ ]:
deduped_df.to_csv(output_path, index=False)
print(f"De-duplicated dataset saved to: {output_path}")